In [1]:
import pennylane as qml
import numpy as np
from pennylane import numpy as pnp
import matplotlib.pyplot as plt
try:
    from jax import numpy as jnp
    import jax
    jax.config.update("jax_enable_x64", True)
except ImportError:
    print("current environment does not have jax installed")

current environment does not have jax installed


In [2]:
from qiskit_ibm_provider import IBMProvider

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options

service = QiskitRuntimeService(channel="ibm_quantum")

## Testing IBMQ

In [4]:
import pennylane as qml
from pennylane_qiskit.qiskit_device2 import QiskitDevice2

use_simulator = True

if use_simulator:
    backend = service.backend("ibmq_qasm_simulator")
    dev = QiskitDevice2(wires=2, backend=backend, use_primitives=True)

    print("RUNNING ON SIMULATOR")
else:
    backend = service.least_busy(simulator=False, operational=True)
    dev = QiskitDevice2(wires=2, backend=backend, use_primitives=True)
    
    print(f"Running on {backend.name}")
    
session = Session(backend=backend)

RUNNING ON SIMULATOR


In [6]:
@qml.qnode(dev)
def circuit(x):
    qml.sum(qml.PauliY(1), qml.PauliZ(0)),
    return qml.expval(qml.PauliZ(0))

circuit(np.pi/2)

(tensor(1., requires_grad=True),)

In [14]:
@qml.qnode(dev)
def circuit_batched(x, y):
    qml.RX([x, y], wires=[0])
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(1)), qml.probs(), qml.var(qml.PauliZ(0)), qml.expval(qml.PauliY(0)), qml.sample()

@qml.qnode(dev)
def circuit(x):
    qml.RX(x, wires=[0])
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(1))

@qml.qnode(dev)
def circuit_counts(x):
    qml.RX(x, wires=[0])
    qml.CNOT(wires=[0, 1])
    return qml.counts()

In [15]:
circuit_batched(0.1, 1.2)

(array([0.99609375, 0.37109375]),
 array([{3: 0.001953125, 0: 0.998046875}, {0: 0.6875, 3: 0.3125}],
       dtype=object),
 array([0.01556396, 0.85345459]),
 array([ 0.00195312, -0.02539062]),
 array([[[1, 1],
         [1, 1],
         [1, 1],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],
 
        [[1, 1],
         [1, 1],
         [1, 1],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]]]))

In [16]:
circuit_batched(0.1, np.pi)

(array([ 0.99414062, -1.        ]),
 array([{3: 0.0009765625, 0: 0.9990234375}, {3: 1.0}], dtype=object),
 array([0.01556396, 0.        ]),
 array([0.015625  , 0.03515625]),
 array([[[1, 1],
         [1, 1],
         [1, 1],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]],
 
        [[1, 1],
         [1, 1],
         [1, 1],
         ...,
         [1, 1],
         [1, 1],
         [1, 1]]]))

In [19]:
from pennylane_qiskit.qiskit_device2 import qiskit_session

angle = 0.1
i = 0

with qiskit_session(dev) as session:
    
    res = circuit(angle)[0]
    print(res)

    while res > 0:
        angle += 0.5
        i += 1
        res = circuit(angle)[0]
        print(res)
        
    print(circuit_counts(angle))
        
i, angle, res

0.994140625
0.8203125
0.458984375
-0.0234375
({'00': 507, '11': 517},)


(3, 1.6, -0.0234375)

## Testing Runtime devices directly in Qiskit for debugging

In [ ]:
from qiskit import QuantumCircuit

# define circuit
def circuit(x):
    qc = QuantumCircuit(2, 2)
    qc.rx(x, 0)
    qc.cx(0, 1)
    qc.measure(1, 0)
    return qc

qc = circuit(1.2)
qc.draw('text')

In [ ]:
from qiskit_ibm_runtime import Sampler

res = []

with Session(backend=backend, max_time="1m"):
    sampler = Sampler()

    for qc in [circuit(0.1), circuit(np.pi/2)]:
        result = sampler.run(qc).result()
        res.append(result.quasi_dists[0])

In [ ]:
from qiskit.primitives import Estimator

estimator = Estimator()
estimator.run?